In [1]:
import pandas as pd
import json
import uuid
from scipy.stats import skewnorm
import numpy as np

In [2]:
# Opción B: con pandas directamente
lugares = pd.read_json("response.json")
# pd.read_json devuelve un dict con clave "results" -> lista de dicts:
lugares = pd.json_normalize(lugares["results"])
lugares.head()

,id,latitud,longitud,name
0,d1db6da4-879c-4996-b1c6-a66b7200d89f,62.065986,-44.807652,Family Pharmacy
1,ffb9d565-7ee3-4bda-aea0-2960c59b1432,73.422899,-40.887461,Tundra Tails
2,856cd043-d865-4658-897b-e8e302ee15b8,67.476237,-34.885311,"Mitchell, Clark and Fox Theater"
3,b29103b7-aeff-4fa1-8232-7088e6bb796c,71.461901,-36.830705,Little Explorers
4,9347d008-c1e8-40b1-81b3-e9e97839228f,74.616162,-27.841708,Glacier Café


In [3]:
coches = pd.read_json("cleaned_Thread-1cars_1501_1506.json")
columns = ["brand", "model", "version", "seatingCapacity", "co2EmissionsGramsPerKm"]
coches = coches[columns]
coches = coches.dropna()
coches = coches.drop_duplicates()
coches.head()

,brand,model,version,seatingCapacity,co2EmissionsGramsPerKm
2,JEEP,Patriot,2.0 CRD Limited,5.0,180.0
4,OPEL,Crossland X,1.2 96kW 130CV Innovation Auto,5.0,113.0
5,SEAT,Alhambra,2.0 TDI 177 CV StartStop Style DSG,5.0,154.0
6,VOLVO,XC90,2.0 T8 AWD Inscription Auto,7.0,50.0
7,LAND-ROVER,Range Rover Sport,3.0 TDV6 258cv HSE,5.0,194.0


In [4]:

def generate_sql_for_coches(df, table_name="coches"):
    """
    Genera el script SQL para crear la tabla y hacer INSERTs de cada fila del DataFrame.
    
    Args:
      df           : pandas.DataFrame con las columnas:
                     'brand', 'model', 'version',
                     'seatingCapacity', 'co2EmissionsGramsPerKm'
      table_name   : nombre de la tabla SQL a crear (por defecto "coches")
    
    Returns:
      Una cadena con todo el script SQL listo para ejecutar.
    """
    # 1) CREATE TABLE
    create = f"""
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

CREATE TABLE IF NOT EXISTS {table_name} (
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    brand VARCHAR(100)     NOT NULL,
    model VARCHAR(100)     NOT NULL,
    version VARCHAR(255),
    seating_capacity INT,
    co2_emissions_grams_per_km INT
);
""".strip()

    # 2) INSERTs
    inserts = []
    for _, row in df.iterrows():
        uid = str(uuid.uuid4())
        # escapamos posibles apóstrofes en texto
        b = row["brand"].replace("'", "''")
        m = row["model"].replace("'", "''")
        v = row["version"].replace("'", "''")
        sc = int(row["seatingCapacity"])
        co2 = int(row["co2EmissionsGramsPerKm"])
        inserts.append(
            f"INSERT INTO {table_name} (id, brand, model, version, seating_capacity, co2_emissions_grams_per_km) "
            f"VALUES ('{uid}', '{b}', '{m}', '{v}', {sc}, {co2});"
        )

    # concatenamos todo
    return "\n\n".join([create] + inserts)

In [5]:
script = generate_sql_for_coches(coches)

# Ahora lo guardas en un fichero
with open("coches.sql", "w", encoding="utf-8") as f:
    f.write(script)

In [6]:
coches = pd.read_json("cars.json")
# pd.read_json devuelve un dict con clave "results" -> lista de dicts:
coches = pd.json_normalize(coches["results"])
coches.head()

,brand,co2_emissions_grams_per_km,id,model,seating_capacity,version
0,JEEP,180,eea0588f-8455-4d06-8c54-f1c0c4b97502,Patriot,5,2.0 CRD Limited
1,OPEL,113,e918a788-a05a-4fc3-8d12-cbaaf1239ba7,Crossland X,5,1.2 96kW 130CV Innovation Auto
2,SEAT,154,4ac6358e-2a45-4761-80f7-4c7d7d731145,Alhambra,5,2.0 TDI 177 CV StartStop Style DSG
3,VOLVO,50,8c867278-dbea-4b3a-8941-5de4f91c672c,XC90,7,2.0 T8 AWD Inscription Auto
4,LAND-ROVER,194,a0ec3952-599c-46d6-95d7-c1b7d798364b,Range Rover Sport,5,3.0 TDV6 258cv HSE


In [7]:
lugares.head()

,id,latitud,longitud,name
0,d1db6da4-879c-4996-b1c6-a66b7200d89f,62.065986,-44.807652,Family Pharmacy
1,ffb9d565-7ee3-4bda-aea0-2960c59b1432,73.422899,-40.887461,Tundra Tails
2,856cd043-d865-4658-897b-e8e302ee15b8,67.476237,-34.885311,"Mitchell, Clark and Fox Theater"
3,b29103b7-aeff-4fa1-8232-7088e6bb796c,71.461901,-36.830705,Little Explorers
4,9347d008-c1e8-40b1-81b3-e9e97839228f,74.616162,-27.841708,Glacier Café


In [8]:
people = pd.read_csv("people.csv")
people.head()

,id,city_id,first_name,last_name,birth_date,job_sector,education,employment_status,income,personality
0,0000013a-0dc5-4a75-838a-faebf8086ae3,564911fc-8df0-4d6c-8ba1-a43f5eeee975,Vladimíra,Křížová,1995-07-27,Education,Masters,Employed,NaN,Determined
1,0000029e-4ae9-43b9-ba43-8462f166b181,39451d17-0e92-4637-b214-80fb0b98ae91,Tamás,Molnár,1949-10-12,Meteorology,High School,Employed,NaN,Progressive
2,0000029f-5de0-425f-a5ef-7cfbac4087c4,ff9dd4a2-1bff-4d25-a32c-c8169f465ad1,Тарас,Наумов,1977-04-09,Government,High School,Unemployed,NaN,Calm
3,000003c9-8743-4247-8c02-4d861c703e20,5cd0d37d-4bb8-430c-9519-d86688e72fa9,Jesse,Wu,1989-11-25,Transportation,High School,Unemployed,NaN,Reserved
4,00000bf3-328e-41d8-b440-af53a4a1c771,70e17f20-af5b-4e74-bc52-19f46e6667fe,Rachel,Simpson,1972-08-23,Food Production,Doctorate,Employed,NaN,Traditional


In [9]:
def create_new_people_columnwise(df: pd.DataFrame, n: int, job_sector: str = "taxi_driver") -> pd.DataFrame:
    """
    Genera n nuevos registros tomando cada campo de forma independiente y aleatoria
    de las columnas del DataFrame original, asignando nuevos UUIDs y forzando
    job_sector a 'taxi_driver'.
    """
    new_data = {}

    for col in df.columns:
        if col == "id":
            # Generar nuevos UUIDs
            new_data[col] = [str(uuid.uuid4()) for _ in range(n)]
        elif col == "job_sector":
            # Forzar job_sector
            new_data[col] = [job_sector] * n
        else:
            # Muestreo aleatorio, respetando valores NA si los hubiera
            # (dropna() opcional si no quieres NAs)
            pool = df[col]
            new_data[col] = pool.sample(n=n, replace=True).values

    return pd.DataFrame(new_data)

def generate_insert_sql_people(df: pd.DataFrame, table_name: str = "people") -> str:
    """
    Dado un DataFrame con las mismas columnas que la tabla 'people',
    construye un bloque de INSERTs en formato SQL para Postgres.
    """
    cols = list(df.columns)
    col_list = ", ".join(cols)
    lines = [f"INSERT INTO {table_name} ({col_list}) VALUES"]

    for i, row in df.iterrows():
        vals = []
        for c in cols:
            v = row[c]
            if pd.isna(v):
                vals.append("NULL")
            elif isinstance(v, (int, float)):
                vals.append(str(v))
            else:
                # escapamos apóstrofes
                txt = str(v).replace("'", "''")
                vals.append(f"'{txt}'")
        sep = "," if i < len(df)-1 else ";"
        lines.append("  (" + ", ".join(vals) + ")" + sep)

    return "\n".join(lines)

# — Ejemplo de uso —
# people = pd.read_csv("people.csv")
# Creamos 10 nuevas personas con job_sector taxi_driver:
new_people = create_new_people_columnwise(people, len(coches))

# Si quieres unirlas al DataFrame original:
people_expanded = pd.concat([people, new_people], ignore_index=True)

# Mostrar las nuevas filas:
new_people.head()

sql_script = generate_insert_sql_people(new_people, table_name="people")

with open("people.sql", "w", encoding="utf-8") as f:
    f.write(sql_script)

In [10]:
taxistas = pd.read_json("taxistas.json")
# pd.read_json devuelve un dict con clave "results" -> lista de dicts:
taxistas = pd.json_normalize(taxistas["results"])
taxistas.head(), 

(                      birth_date                               city_id  \
 0  Fri, 22 Jul 1966 00:00:00 GMT  d357124e-ec8e-46cb-85b4-2d6df6748682   
 1  Fri, 16 Mar 1973 00:00:00 GMT  112e2476-e9d6-42bc-94f2-c9fa10b29310   
 2  Thu, 08 Jul 1976 00:00:00 GMT  9a0d211d-47c0-48c0-8280-da6e171fc32f   
 3  Sun, 04 Dec 1988 00:00:00 GMT  083a1ed4-8341-4cb0-b8f6-031c76f5b46f   
 4  Mon, 01 Feb 1999 00:00:00 GMT  df7b67ea-5b18-4a88-85a4-8fa5409f3009   
 
      education employment_status first_name  \
 0      Masters          Employed       Mark   
 1    Bachelors          Employed   Benjamin   
 2    Bachelors        Unemployed   Tillmann   
 3    Bachelors        Unemployed   Jonathan   
 4  High School          Employed        Jon   
 
                                      id income   job_sector  last_name  \
 0  12c11e26-e54a-4474-86af-425662c7e54c   None  taxi_driver  Johansson   
 1  7f723c38-7910-43fd-bb45-889b7573e89d   None  taxi_driver     Morgan   
 2  7891bc75-add8-4a80-b994-8d2ca

In [11]:
tabla_taxistas = pd.DataFrame({"people_id": people["id"], "car_id": coches["id"], "co2EmissionsGramsPerKm": coches["co2_emissions_grams_per_km"]})

In [12]:
tabla_taxistas.head()

,people_id,car_id,co2EmissionsGramsPerKm
0,0000013a-0dc5-4a75-838a-faebf8086ae3,eea0588f-8455-4d06-8c54-f1c0c4b97502,180
1,0000029e-4ae9-43b9-ba43-8462f166b181,e918a788-a05a-4fc3-8d12-cbaaf1239ba7,113
2,0000029f-5de0-425f-a5ef-7cfbac4087c4,4ac6358e-2a45-4761-80f7-4c7d7d731145,154
3,000003c9-8743-4247-8c02-4d861c703e20,8c867278-dbea-4b3a-8941-5de4f91c672c,50
4,00000bf3-328e-41d8-b440-af53a4a1c771,a0ec3952-599c-46d6-95d7-c1b7d798364b,194


In [13]:
def generate_sql_for_taxistas(df, table_name="taxi_drivers", sql_path="taxistas.sql"):
    """
    Genera un script SQL con:
      1) CREATE TABLE
      2) INSERTs de cada fila de df
    y lo guarda en sql_path.
    """
    # 1) DDL
    create = f"""
-- Habilitar extensión de UUID si hiciese falta
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

CREATE TABLE IF NOT EXISTS {table_name} (
    people_id UUID NOT NULL,
    car_id    UUID NOT NULL,
    co2_emissions_grams_per_km INT,
    PRIMARY KEY (people_id, car_id),
    FOREIGN KEY (people_id) REFERENCES people(id),
    FOREIGN KEY (car_id)    REFERENCES cars(id)
);
""".strip()

    # 2) INSERTs
    inserts = ["\n-- INSERTs de registros\n"]
    cols = ["people_id", "car_id", "co2_emissions_grams_per_km"]
    for _, row in df.iterrows():
        ppl = row["people_id"]
        car = row["car_id"]
        co2 = row["co2EmissionsGramsPerKm"]
        val_co2 = "NULL" if pd.isna(co2) else int(co2)
        inserts.append(
            f"INSERT INTO {table_name} ({', '.join(cols)}) "
            f"VALUES ('{ppl}', '{car}', {val_co2});"
        )

    # 3) Volcar a fichero
    with open(sql_path, "w", encoding="utf-8") as f:
        f.write(create + "\n\n" + "\n".join(inserts))

    print(f"Script SQL guardado en: {sql_path}")
    

generate_sql_for_taxistas(tabla_taxistas.dropna())

Script SQL guardado en: taxistas.sql


In [14]:
def muestrear_skewed(lista, tamaño_muestra, sesgo=0.0, with_replacement=True):
    """
    Devuelve una lista de longitud `tamaño_muestra` muestreada de `lista`
    siguiendo una distribución skew-normal.

    Parámetros
    ----------
    lista : iterable de números
        Los valores posibles a muestrear.
    tamaño_muestra : int
        Cuántos elementos devolver.
    sesgo : float, opcional
        Parámetro 'a' de skewnorm:
          - sesgo > 0 ⇒ cola a la derecha (más valores pequeños)
          - sesgo < 0 ⇒ cola a la izquierda (más valores grandes)
        Por defecto 0 (distribución normal simétrica).
    with_replacement : bool, opcional
        Si True (por defecto), muestreo con reemplazo.
        Si False, muestreo sin reemplazo (requiere tamaño_muestra ≤ len(lista)).

    Retorna
    -------
    list
        Lista de elementos seleccionados.
    """
    arr = np.array(lista, dtype=float)
    # Ajustamos ubicación y escala a los de los datos originales
    loc  = arr.mean()
    scale = arr.std(ddof=0) if arr.std(ddof=0) > 0 else 1.0

    # 1. Calculamos la densidad en cada punto
    dens = skewnorm.pdf(arr, a=sesgo, loc=loc, scale=scale)

    # 2. Convertimos densidades en probabilidades
    probs = dens / np.sum(dens)

    # 3. Muestreamos índices según esas probabilidades
    indices = np.random.choice(
        np.arange(len(arr)),
        size=tamaño_muestra,
        replace=with_replacement,
        p=probs
    )

    return arr[indices].tolist()

In [15]:
number_of_trips = 40000

In [16]:
destinys = muestrear_skewed(lista=range(0, 100),
                            tamaño_muestra = number_of_trips,
                            sesgo=0.6
                            )

origins = muestrear_skewed(lista=range(0, 100),
                            tamaño_muestra = number_of_trips,
                            sesgo=-0.6
                            )

len(destinys), len(origins)

(40000, 40000)

In [17]:
trips = pd.DataFrame({
    "origin":  lugares["id"].iloc[origins].values,
    "destiny": lugares["id"].iloc[destinys].values
})
len(trips)

40000

In [18]:
trips_count = (
    trips
    .groupby(["origin", "destiny"])
    .size()                  # count rows per group
    .reset_index(name="count")
)

# 2) Sort descending by count
trips_count = trips_count.sort_values("count", ascending=False)

# 3) Show the top N most-repeated pairs
top_n = 10
print(trips_count.head(top_n))

freq = trips_count["count"] \
          .value_counts() \
          .sort_index()  # ordena por el número de viajes
print(freq)

                                    origin  \
7392  e90a4b5e-4279-4cbe-9e9d-e94fa3c776e5   
4393  a5c7cefc-0b22-462f-9b40-a3e3dc1e0f89   
7690  ed3865ed-84ff-48c6-843f-0f0d4c999aab   
4191  9e8d5c96-29ea-4242-b7f1-685cdd350ce2   
5051  b7672b8e-48f5-47d2-b92a-87e71d5f3966   
6806  e08d7dff-9423-448e-8c17-8a9923b85c9d   
429   1f6f8f1a-cbcb-4577-bd0a-99250ca9fd24   
2828  7fa79250-bc77-4cbd-9321-5fafc5650eb1   
1797  53599b5f-db7b-4962-9cbb-47010ea88933   
7795  ee18efd3-7d0f-499a-8a7e-edad3a59eab2   

                                   destiny  count  
7392  316f2161-09d9-445c-82cc-f18116e5ccaa     22  
4393  faa1c66c-e4ec-4b73-91f6-6cdf496dd027     20  
7690  a6be05d9-7bab-4512-b29e-83c9f70d00e4     20  
4191  dace8005-8e52-47c9-b928-130cbc63a127     19  
5051  c61a7503-2e18-48d7-94de-d33e43bf33e4     19  
6806  6095aa0b-f110-4d3b-9b1c-43cf6de0f6f1     19  
429   c0fbcf41-e181-4c4d-b514-b36cc7c9d056     19  
2828  9e0ea558-2c56-4742-91e6-677cc90f0658     19  
1797  c4226a9f-3012-4407-

In [19]:
trips.head()

,origin,destiny
0,bdfe24ab-7548-4285-8df1-e37cb3555615,9739f2fd-ff38-4bce-a2db-44d01fc31434
1,df649815-15de-4dc2-971a-f458dc602b8b,fabc4719-7df2-4cf8-a202-773322deb05b
2,e64cf4f2-2f79-4885-a0fc-4bbb8219abc2,bc9b9d21-b057-46ae-95a8-417748786dbe
3,e64cf4f2-2f79-4885-a0fc-4bbb8219abc2,c0fbcf41-e181-4c4d-b514-b36cc7c9d056
4,a0e9a28b-98ad-49bb-a98d-41d3059f6ac1,ed368bb4-c5ab-4b6a-8770-6c3674cc01d6


In [20]:
taxistas.head()

,birth_date,city_id,education,employment_status,first_name,id,income,job_sector,last_name,personality
0,"Fri, 22 Jul 1966 00:00:00 GMT",d357124e-ec8e-46cb-85b4-2d6df6748682,Masters,Employed,Mark,12c11e26-e54a-4474-86af-425662c7e54c,None,taxi_driver,Johansson,Disciplined
1,"Fri, 16 Mar 1973 00:00:00 GMT",112e2476-e9d6-42bc-94f2-c9fa10b29310,Bachelors,Employed,Benjamin,7f723c38-7910-43fd-bb45-889b7573e89d,None,taxi_driver,Morgan,Resourceful
2,"Thu, 08 Jul 1976 00:00:00 GMT",9a0d211d-47c0-48c0-8280-da6e171fc32f,Bachelors,Unemployed,Tillmann,7891bc75-add8-4a80-b994-8d2ca22cbc82,None,taxi_driver,Guillon,Traditional
3,"Sun, 04 Dec 1988 00:00:00 GMT",083a1ed4-8341-4cb0-b8f6-031c76f5b46f,Bachelors,Unemployed,Jonathan,79a2405c-3a65-4ac5-a417-f9d2e40ce57c,None,taxi_driver,Johansson,Disciplined
4,"Mon, 01 Feb 1999 00:00:00 GMT",df7b67ea-5b18-4a88-85a4-8fa5409f3009,High School,Employed,Jon,7e377f00-f385-4a1b-b4ec-07bc756f589b,None,taxi_driver,Jokinen,Disciplined


In [21]:
taxistas

,birth_date,city_id,education,employment_status,first_name,id,income,job_sector,last_name,personality
0,"Fri, 22 Jul 1966 00:00:00 GMT",d357124e-ec8e-46cb-85b4-2d6df6748682,Masters,Employed,Mark,12c11e26-e54a-4474-86af-425662c7e54c,None,taxi_driver,Johansson,Disciplined
1,"Fri, 16 Mar 1973 00:00:00 GMT",112e2476-e9d6-42bc-94f2-c9fa10b29310,Bachelors,Employed,Benjamin,7f723c38-7910-43fd-bb45-889b7573e89d,None,taxi_driver,Morgan,Resourceful
2,"Thu, 08 Jul 1976 00:00:00 GMT",9a0d211d-47c0-48c0-8280-da6e171fc32f,Bachelors,Unemployed,Tillmann,7891bc75-add8-4a80-b994-8d2ca22cbc82,None,taxi_driver,Guillon,Traditional
3,"Sun, 04 Dec 1988 00:00:00 GMT",083a1ed4-8341-4cb0-b8f6-031c76f5b46f,Bachelors,Unemployed,Jonathan,79a2405c-3a65-4ac5-a417-f9d2e40ce57c,None,taxi_driver,Johansson,Disciplined
4,"Mon, 01 Feb 1999 00:00:00 GMT",df7b67ea-5b18-4a88-85a4-8fa5409f3009,High School,Employed,Jon,7e377f00-f385-4a1b-b4ec-07bc756f589b,None,taxi_driver,Jokinen,Disciplined
...,...,...,...,...,...,...,...,...,...,...
267,"Mon, 02 Nov 1942 00:00:00 GMT",70e17f20-af5b-4e74-bc52-19f46e6667fe,High School,Employed,Jared,0efa539a-8cd6-4201-bb61-207c92d49bb8,None,taxi_driver,Wesack,Resourceful
268,"Sat, 14 Dec 1996 00:00:00 GMT",9c8f9bbf-45c1-4757-82fb-df4fd6b7b533,Bachelors,Employed,John,d050b497-ef00-440d-9bac-d4536c85ae92,None,taxi_driver,Stiffel,Traditional
269,"Wed, 08 Jan 2003 00:00:00 GMT",bd7255a2-1274-4496-a1fa-5045a429bebb,Bachelors,Unemployed,Jon,4af209f7-4883-42cb-86e8-ac6d4379bca3,None,taxi_driver,Guillen,Disciplined
270,"Fri, 10 Sep 1965 00:00:00 GMT",9a4ac49e-0928-40ce-a950-5e2119868abf,High School,Unemployed,Beatrice,cc2b04ad-56b2-490c-9b15-87af263671d6,None,taxi_driver,Da Costa,Calm


In [22]:
trips

,origin,destiny
0,bdfe24ab-7548-4285-8df1-e37cb3555615,9739f2fd-ff38-4bce-a2db-44d01fc31434
1,df649815-15de-4dc2-971a-f458dc602b8b,fabc4719-7df2-4cf8-a202-773322deb05b
2,e64cf4f2-2f79-4885-a0fc-4bbb8219abc2,bc9b9d21-b057-46ae-95a8-417748786dbe
3,e64cf4f2-2f79-4885-a0fc-4bbb8219abc2,c0fbcf41-e181-4c4d-b514-b36cc7c9d056
4,a0e9a28b-98ad-49bb-a98d-41d3059f6ac1,ed368bb4-c5ab-4b6a-8770-6c3674cc01d6
...,...,...
39995,3d21491c-ed5d-42d0-ba57-4ace47920774,900ba99d-1ae2-4e7a-9479-e9c36dd80cb3
39996,9e811f91-3f46-498d-b17d-f319951fe020,e4551865-a86a-4154-82d0-b1ec0f639c9f
39997,df649815-15de-4dc2-971a-f458dc602b8b,bc9b9d21-b057-46ae-95a8-417748786dbe
39998,ed3865ed-84ff-48c6-843f-0f0d4c999aab,316f2161-09d9-445c-82cc-f18116e5ccaa


In [23]:
taxi_ids = tabla_taxistas["people_id"].dropna().values

# 2) Muestreamos uno para cada viaje (len(trips)), con reemplazo
assigned = np.random.choice(taxi_ids, size=len(trips), replace=True)

# 3) Añadimos la columna al DataFrame de trips
trips["taxista_id"] = assigned

In [27]:
trips.head()
len(trips)

40000

In [ ]:


# Veamos las primeras fila
trips_full = trips

KeyError: 'taxista_id'

In [ ]:

# Creamos una lista de UUID4, uno por cada fila
trip_ids = [str(uuid.uuid4()) for _ in range(len(trips_full))]

# La añadimos como nueva columna
trips_full["trip_id"] = trip_ids

# Opcional: movemos la nueva columna al principio
cols = ["trip_id"] + [c for c in trips_full.columns if c != "trip_id"]
trips_full = trips_full[cols]
trips_full.head()

,trip_id,origin,destiny,taxista_id


In [ ]:
# 1) Calcula la frecuencia de cada viaje
trips_full['freq'] = trips_full.groupby(['origin','destiny'])['trip_id'] \
                               .transform('count')

# 2) Normaliza esa frecuencia al rango [0, 1]
fmin, fmax = trips_full['freq'].min(), trips_full['freq'].max()
trips_full['freq_norm'] = (trips_full['freq'] - fmin) / (fmax - fmin)

# 3) Define el rango de km que quieres asignar
min_km, max_km = 5, 30

# 4) Genera la columna 'km' como valor lineal según freq_norm
trips_full['km'] = trips_full['freq_norm'] * (max_km - min_km) + min_km

# (Opcional) Redondea a un decimal
trips_full['km'] = trips_full['km'].round(1)

# 5) (Opcional) Limpia columnas auxiliares
trips_full = trips_full.drop(columns=['freq','freq_norm'])

/tmp/ipykernel_50690/1679582579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_full['freq'] = trips_full.groupby(['origin','destiny'])['trip_id'] \
/tmp/ipykernel_50690/1679582579.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_full['freq_norm'] = (trips_full['freq'] - fmin) / (fmax - fmin)
/tmp/ipykernel_50690/1679582579.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [ ]:
trips_full

,trip_id,origin,destiny,taxista_id,km
0,13180dbf-3e1a-4b6b-a206-0831ef38c02b,3956eb61-e388-4a94-8b56-c00b4b669104,a6be05d9-7bab-4512-b29e-83c9f70d00e4,98f83036-af49-4e4c-899c-be8b49fc3a26,13.8
1,3608bd53-e53d-42f5-9828-b39b4ce8a77c,e64cf4f2-2f79-4885-a0fc-4bbb8219abc2,69cacb4b-b212-442f-aa4e-7004f5b2d40a,791f1bcf-8d3a-401f-b7c6-fa8b97b74b28,10.0
2,4e8b43aa-ac05-421c-bf6c-a90f7ad39c1e,5291b446-7d17-4649-bd60-fdbb73bb0591,939517d3-f9b1-4927-968b-6b5557d5b810,901f12fb-cbd8-40ae-adc8-a742ab83bdd3,10.0
3,b50de504-20c9-4114-b09b-4c8a8d18b526,ed3865ed-84ff-48c6-843f-0f0d4c999aab,037c3a71-aeb6-4b6c-8778-c70baeeb3770,a4e7e2dd-6a1e-44fe-8127-74761471dc3d,8.8
4,f24c11ce-a17f-4fdb-b245-e3dde6ed2a2f,c6d9848a-3e9c-48dd-b744-682f0a8e2256,f309cb82-a588-433a-917f-ea008f24ba81,0ac5f657-df67-4d98-9893-668c1e673a56,20.0
...,...,...,...,...,...
39995,8497161e-e591-4f14-a380-abf876d0c334,e4551865-a86a-4154-82d0-b1ec0f639c9f,faa1c66c-e4ec-4b73-91f6-6cdf496dd027,79414551-7303-4b01-8d0a-4250fc680902,10.0
39996,3c6f5103-b1bf-4544-81b6-d657de68c935,bdfe24ab-7548-4285-8df1-e37cb3555615,3d21491c-ed5d-42d0-ba57-4ace47920774,485bfbb5-63a0-487b-8f4c-231eb6e4076b,6.2
39997,fcf77ec1-7dae-4134-9592-44e410911d10,9a682bc5-7456-4932-8ae3-5079fa6005bc,9e8d5c96-29ea-4242-b7f1-685cdd350ce2,329db10c-d478-4ca2-b20c-9511c2c74d2a,20.0
39998,c7e2d287-0fe7-4a60-ba7e-19d5660a793a,e15586fc-b514-4862-81cd-16b42c628c5d,316f2161-09d9-445c-82cc-f18116e5ccaa,c8c64307-7f93-4059-99bc-51a64ae56f83,5.0


In [ ]:
def generate_inserts(df: pd.DataFrame, table_name: str = "trips_full") -> str:
    """
    Genera un bloque de INSERTs para PostgreSQL a partir de un DataFrame.
    Devuelve un único string con todos los INSERTs separados por salto de línea.
    """
    inserts = []
    for _, row in df.iterrows():
        trip_id, origin, destiny, taxista_id, km = (
            row["trip_id"],
            row["origin"],
            row["destiny"],
            row["taxista_id"],
            row["km"],
        )
        # Escapamos apóstrofes si los hubiera (aunque aquí trabajamos con UUIDs)
        template = (
            f"INSERT INTO {table_name} (trip_id, origin, destiny, taxista_id, km) "
            f"VALUES ('{trip_id}', '{origin}', '{destiny}', '{taxista_id}', {km});"
        )
        inserts.append(template)
    # Devolvemos todos los INSERTs unidos por salto de línea
    return "\n".join(inserts)



In [ ]:
sql_script = generate_inserts(trips_full, table_name="trips_full")
    # Ahora puedes escribir sql_script a un archivo .sql o ejecutarlo directamente
with open("inserts_trips_full.sql", "w", encoding="utf-8") as f:
    f.write(sql_script)
print("Archivo inserts_trips_full.sql generado con éxito.")

Archivo inserts_trips_full.sql generado con éxito.
